# Company Email Project

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle


## Company Emails
For this project I worked with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [4]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Salary Prediction

Using network `G`, I identified the people in the network with missing values for the node attribute `ManagementSalary` and predicted whether or not these individuals are receiving a managment position salary.

To accomplish this, I needed to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



My predictions are given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).


In [5]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [24]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.svm import SVC
    from sklearn.metrics import confusion_matrix, classification_report
# Create dataframe of relevant data   
    df = pd.DataFrame(index=G.nodes)
    df.index.names = ['Node']
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    # df['clustering'] = pd.Series(nx.clustering(G))
    df['degree centrality'] = pd.Series(nx.degree_centrality(G))
    df["shortest path"] = pd.Series(list(nx.shortest_path(G)))
# Split dataframe on training data for random forest classifier and data that needs to be classified
    df_train = df[df['ManagementSalary'].notnull()]
    df_test = df[df['ManagementSalary'].isnull()]
# Create X and y for training classifier
    X = df_train.drop('ManagementSalary', axis = 1)
    y = df_train['ManagementSalary']
# Train test split
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.fit_transform(X_test)
# # Train model
    rfc = RandomForestClassifier(max_depth = 5)
    rfc.fit(X_train, y_train)
    # rfpredictions = rfc.predict(X_test)
    rfc_auc = roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1])
# # Apply Model
    df_test = df_test.drop('ManagementSalary', axis = 1)
    # df_test['ManagementSalary'] = pd.Series(rfc.predict(df_test))
    df_test_tran = ss.fit_transform(df_test)
    answer = pd.Series(rfc.predict_proba(df_test_tran)[:, 1], index= df_test.index)
    return answer, "the AUC score is: "+str(rfc_auc)
    # return rfc.predict_proba(df_test)[:, 1]
salary_predictions()

(Node
 1      0.293804
 65     0.490633
 18     0.092567
 215    0.251156
 283    0.932332
          ...   
 691    0.060019
 788    0.100908
 944    0.100429
 798    0.021664
 808    0.055189
 Length: 252, dtype: float64,
 'the AUC score is: 0.8564693556836039')

### New Connections Prediction

I predicted future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [26]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, I identified the edges in `future_connections` with missing values and predicted whether or not these edges will have a future connection.



My predictions are given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).


In [27]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
#   create features for classifier
    future_connections['common_neighbors'] = [len(list(nx.common_neighbors(G, edge[0],edge[1]))) for edge in future_connections.index]
    future_connections["preferential_attachment"] = [i[2] for i in nx.preferential_attachment(G, ebunch=future_connections.index)]
    future_connections["jaccard"] = [i[2] for i in nx.jaccard_coefficient(G, ebunch=future_connections.index)]
# Split data for training and predicting
    df_train = future_connections[~future_connections['Future Connection'].isnull()]
    df_test = future_connections[future_connections['Future Connection'].isnull()]
# Create training data
    X = df_train.drop('Future Connection', axis = 1)
    y = df_train['Future Connection']
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.fit_transform(X_test)
# train random forest classifier with training data
    rfc = RandomForestClassifier(max_depth = 5)
    rfc.fit(X_train, y_train)
# Compute Area under the curve score
    rfc_auc = roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1])
# # Apply Model
    df_test = df_test.drop('Future Connection', axis = 1)
    df_test_tran = ss.fit_transform(df_test)
    answer = pd.Series(rfc.predict_proba(df_test_tran)[:, 1], index= df_test.index)
    return answer, "the AUC score is: "+str(rfc_auc)

new_connections_predictions()
    

((107, 348)    0.023702
 (542, 751)    0.014528
 (20, 426)     0.605238
 (50, 989)     0.014041
 (942, 986)    0.014007
                 ...   
 (165, 923)    0.016813
 (673, 755)    0.014042
 (939, 940)    0.014007
 (555, 905)    0.014739
 (75, 101)     0.018963
 Length: 122112, dtype: float64,
 'the AUC score is: 0.9084432914858716')